# Linear Search

go through all elements to find the target

In [3]:
class LinearSearch:
    def search(self, data: list[int], target: int) -> int:
        result = -1
        
        left = 0
        right = len(data) - 1
        while left <= right:
            if data[left] == target:
                result = left
                break
            
            if data[right] == target:
                result = right
                break
                
            left += 1
            right -= 1
                
        return result

In [30]:
import time
import random

data_len = 1000
data = [random.randint(0, 999) for i in range(data_len)]

# test range
range_start = time.perf_counter()
for i in range(0, data_len):
    range_val = data[i]
range_end = time.perf_counter()
range_perf = range_end - range_start

# test enumerate
enum_start = time.perf_counter()
for i, v in enumerate(data):
    enum_val = v
enum_end = time.perf_counter()
enum_perf = enum_end - enum_start

print("range spent {}S, enum spent {}S".format(range_perf, enum_perf))

range spent 0.00019868900017172564S, enum spent 0.00018150500000047032S


# Binary Search

<p style="color:red">*for sorted data</p>
recursively half the data until find the target

In [5]:
class BinarySearch:
    def search(self, data: list[int], target: int) -> int:
        data_len = len(data)
        idx_list = list(range(data_len))
        
        result = self.compare(data, idx_list, target)
        return result
        
    def compare(self, data: list[int], idx_list: list[int], target: int) -> int:
        idx_list_len = len(idx_list)
        
        if idx_list_len == 0:
            return -1
        
        if idx_list_len == 1:
            return idx_list[0] if data[idx_list[0]] == target else -1
        
        mid_idx = idx_list_len // 2
        mid_idx_val = idx_list[mid_idx]
        idx_list = idx_list[0:mid_idx] if target < data[mid_idx_val] else idx_list[mid_idx:idx_list_len]
        return self.compare(data, idx_list, target)
        
    # from https://www.geeksforgeeks.org/binary-search/
    def better_sol(self, data: list[int], left: int, right: int, target: int) -> int:
        if right >= left:
            # add left upon the middle of the rest space to get the exact middle of each run
            mid = left + (right - left) // 2
            
            if data[mid] == target:
                return mid
            
            if data[mid] > target:
                return self.better_sol(data, left, mid - 1, target)
            
            if data[mid] < target:
                return self.better_sol(data, mid + 1, right, target)
        
        else:
            return -1

# Jump Search

<p style="color:red">*for sorted data</p>
recursively compare the element of jumped index with the target until the value is larger than the target, then apply linear search backward toward the previous jumped item

In [6]:
import math

class JumpSearch:
    def search(self, data: list[int], target: int) -> int:
        data_len = len(data)
        step = int(math.sqrt(data_len))
        idx = 0
        
        # 2022-05-16: new code inspired from expontential search approach
        # check data[0] before jump
        if data[idx] == target:
            return 0
        
        # jump
        while idx < data_len and data[idx] < target:
            idx += step
        # set left = previous step idx, right = current idx or last index 
        left = idx // 2
        right = min(idx, data_len - 1)
        return self.ln_search(data, left, right, target)
        
        # jump_idx = 0
        
        # # jump until the last block
        # while jump_idx < data_len:
        #     if data[jump_idx] >= target:
        #         return self.ln_search(data, jump_idx - step, jump_idx, target)
        #     jump_idx += step
            
        # # last block: from the last step to len(data), if no match return -1
        # return self.ln_search(data, jump_idx - step, data_len - 1, target)
                
    def ln_search(self, data: list[int], start: int, end: int, target) -> int:
        while start <= end:
            if data[start] == target:
                return start
            start += 1
        return -1

# Interpolation Search

<p style="color:red">*for sorted data <b>with uniformly distributed values (linear distribution)</b></p>
to deal with linear distributed data case, we can refer to binary search strategy, dividing the data into the pos item, left part, and right part. but instead of choosing the middle of data as split index, we apply linear formula to get the pos which will closer to left or right part depends on the left and right indexes

In [7]:
class InterpolationSearch:
    def search(self, data: list[int], left: int, right: int, target: int) -> int:
        if (left <= right and target >= data[left] and target <= data[right]):
            
            # Let's assume that the elements of the array are linearly distributed.
            # General equation of line: y = m*x + c.
            # y is the value in the array and x is its index.

            # Now putting value of lo, hi and x in the equation
            # arr[hi] = m*hi+c - ---(1)
            # arr[lo] = m*lo+c - ---(2)
            # x = m*pos + c - ---(3)

            # m = (arr[hi] - arr[lo]) / (hi - lo)

            # subtracting eqxn(2) from (3)
            # x - arr[lo] = m * (pos - lo)
            # lo + (x - arr[lo])/m = pos
            # pos = lo + (x - arr[lo]) * (hi - lo)/(arr[hi] - arr[lo])
            pos = left + (target - data[left]) * (right - left) // (data[right] - data[left])
            
            if data[pos] == target:
                return pos
            
            # target index must between pos+1 and right end
            if data[pos] < target:
                return self.search(data, pos + 1, right, target)
            
            # target index must between left end and pos-1
            if data[pos] > target:
                return self.search(data, left, pos -1, target)
            
        return -1

# Exponential Search

<p style="color:red">*for sorted data</p>

In [8]:
class ExponentialSearch:
    def search(self, data: list[int], target: int) -> int:
        data_len = len(data)
        idx = 0
        if data[idx] == target:
            return 0
        
        idx = 1
        while idx < data_len and data[idx] < target:
            idx *= 2
        
        binary_search = BinarySearch()
        return binary_search.better_sol(data, idx // 2, min(idx, data_len - 1), target)

# Test

In [21]:
import time
import random
# test_data = [1, 5, 8, 9, 11, 16, 17, 18, 20, 21]
# set() -> unique values
# sorted() -> sorted list
test_data = sorted(list(set(random.randint(0, 99) for i in range(100))))

test_target = 25
# print("test data: {} \ntarget value: {}".format(test_data, test_target))

# linear research
linear_search = LinearSearch()
linear_search_s = time.perf_counter()
linear_search_result = linear_search.search(test_data, test_target)
linear_search_e = time.perf_counter()
linear_search_perf = linear_search_e - linear_search_s
print("\nLinear search complete, target index: {} \nTime spent: {}".format(
    linear_search_result, linear_search_perf))

# binary research
binary_search = BinarySearch()
binary_search_s = time.perf_counter()
# binary_search_result = binary_search.search(test_data, test_target) #original
binary_search_result = binary_search.better_sol(test_data, 0, len(test_data) - 1, test_target) #sample
binary_search_e = time.perf_counter()
binary_search_perf = binary_search_e - binary_search_s
print("\nBinary search complete, target index: {} \nTime spent: {}".format(
    binary_search_result, binary_search_perf))

# jump research
jump_search = JumpSearch()
jump_search_s = time.perf_counter()
jump_search_result = jump_search.search(test_data, test_target)
jump_search_e = time.perf_counter()
jump_search_perf = jump_search_e - jump_search_s
print("\nJump search complete, target index: {} \nTime spent: {}".format(
    jump_search_result, jump_search_perf))

# interpolation research
interpolation_search = InterpolationSearch()
interpolation_search_s = time.perf_counter()
interpolation_search_result = interpolation_search.search(test_data, 0, len(test_data) - 1, test_target)
interpolation_search_e = time.perf_counter()
interpolation_search_perf = interpolation_search_e - interpolation_search_s
print("\nInterpolation search complete, target index: {} \nTime spent: {}".format(
    interpolation_search_result, interpolation_search_perf))

# expontential research
exponential_search = ExponentialSearch()
exponential_search_s = time.perf_counter()
exponential_search_result = exponential_search.search(test_data, test_target)
exponential_search_e = time.perf_counter()
exponential_search_perf = exponential_search_e - exponential_search_s
print("\nExponential search complete, target index: {} \nTime spent: {}".format(
    exponential_search_result, exponential_search_perf))





Linear search complete, target index: 17 
Time spent: 7.65310005590436e-05

Binary search complete, target index: 17 
Time spent: 8.693000017956365e-05

Jump search complete, target index: 17 
Time spent: 7.6231000093685e-05

Interpolation search complete, target index: 17 
Time spent: 0.0001289470001211157

Exponential search complete, target index: 17 
Time spent: 7.877000007283641e-05
